In [1]:
#IMPORTACIÓN DE LIBRERÍAS

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from string import ascii_uppercase as alfabeto
import pickle
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

## **Datos referentes al ranking FIFA**

In [5]:
ranking = pd.read_html("https://en.wikipedia.org/wiki/FIFA_Men's_World_Ranking")
historical_ranking = ranking[3]
actual_ranking = ranking[0]

In [6]:
actual_ranking = actual_ranking.drop(0)
actual_ranking = actual_ranking.drop(1)
actual_ranking = actual_ranking.drop(2)
actual_ranking = actual_ranking.drop(23)
actual_ranking = actual_ranking.drop(24)

In [7]:
actual_ranking

,0,1,2,3
3,1,NaN,Argentina,1901.48
4,2,NaN,France,1854.91
5,3,5,Spain,1835.67
6,4,1,England,1812.26
7,5,1,Brazil,1785.61
8,6,3,Belgium,1772.44
9,7,NaN,Netherlands,1758.51
10,8,2,Portugal,1741.43
11,9,3,Colombia,1727.32
12,10,NaN,Italy,1714.29


In [8]:
actual_ranking.rename(columns={actual_ranking.columns[0]: 'Rank'}, inplace=True)
actual_ranking.rename(columns={actual_ranking.columns[1]: 'Change'}, inplace=True)
actual_ranking.rename(columns={actual_ranking.columns[2]: 'Team'}, inplace=True)
actual_ranking.rename(columns={actual_ranking.columns[3]: 'Points'}, inplace=True)

In [9]:
actual_ranking

,Rank,Change,Team,Points
3,1,NaN,Argentina,1901.48
4,2,NaN,France,1854.91
5,3,5,Spain,1835.67
6,4,1,England,1812.26
7,5,1,Brazil,1785.61
8,6,3,Belgium,1772.44
9,7,NaN,Netherlands,1758.51
10,8,2,Portugal,1741.43
11,9,3,Colombia,1727.32
12,10,NaN,Italy,1714.29


In [83]:
actual_ranking.pop('Change')

3     NaN
4     NaN
5       5
6       1
7       1
8       3
9     NaN
10      2
11      3
12    NaN
13      3
14      3
15      3
16      2
17      4
18      5
19      2
20      1
21      1
22    NaN
Name: Change, dtype: object

In [84]:
actual_ranking

,Rank,Team,Points
3,1,Argentina,1901.48
4,2,France,1854.91
5,3,Spain,1835.67
6,4,England,1812.26
7,5,Brazil,1785.61
8,6,Belgium,1772.44
9,7,Netherlands,1758.51
10,8,Portugal,1741.43
11,9,Colombia,1727.32
12,10,Italy,1714.29


## **Recolección de datos inicial con bs4**

In [2]:
#Cargamos los datos de la página html

tablas = pd.read_html('https://en.wikipedia.org/wiki/2024_Copa_America')
#tablas_mundial = pd.read_html('https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')
# ra = pd.read_html('https://es.wikipedia.org/wiki/Clasificación_mundial_de_la_FIFA')

In [3]:
#Encontramos un patrón dentro de las tablas: tablas[14], tablas[21], tablas[28], tablas[35]
#A partir del índice 14, y aumentando de 7 en 7, obtenemos las tablas de los cuatro grupos de la Copa
tablas[35]

,Pos,Teamvte,Pld,W,D,L,GF,GA,GD,Pts,Qualification
0,1,Colombia,3,2,1,0,6,2,+4,7,Advance to knockout stage
1,2,Brazil,3,1,2,0,5,2,+3,5,Advance to knockout stage
2,3,Costa Rica,3,1,1,1,2,4,−2,4,NaN
3,4,Paraguay,3,0,0,3,3,8,−5,0,NaN


In [4]:
#Creamos el DataFrame con los grupos de la copa actual

dict_tables = {} #Creamos un diccionario vacío para rellenarlo con las tablas
for letra, i in zip(alfabeto, range(14, 42, 7)):
    df = tablas[i]
    df.rename(columns={df.columns[1]: 'Team'}, inplace=True)
    df.pop('Qualification')
    dict_tables[f'Group {letra}'] = df

In [5]:
#Probamos para el grupo D
dict_tables['Group D']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Colombia,3,2,1,0,6,2,+4,7
1,2,Brazil,3,1,2,0,5,2,+3,5
2,3,Costa Rica,3,1,1,1,2,4,−2,4
3,4,Paraguay,3,0,0,3,3,8,−5,0


In [6]:
#Creamos una lista que contiene todas las ediciones de la Copa América

years_America = [1916, 1917, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1929, 1935, 1937, 1939, 1941,
                 1942, 1945, 1946, 1947, 1949, 1953, 1955, 1956, 1957, 1963, 1967, 1975, 1979, 1983, 1987, 1989,
                 1991, 1993, 1995, 1997, 1999, 2001, 2004, 2007, 2011, 2015, 2016, 2019, 2021, 2024]

years_World = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974, 1978, 1982, 1986, 1990, 1994, 1998,
               2002, 2006, 2010, 2014, 2018, 2022, 2026]

In [7]:
#WebScrapping con BeautifulSoup

def get_matches(year_A):
    
    #Se crea el condicional para los dos posibles formatos de link a wiki
    if year_A <= 1967:
        url = f'https://en.wikipedia.org/wiki/{year_A}_South_American_Championship'
    else:
        url = f'https://en.wikipedia.org/wiki/{year_A}_Copa_America'
    
    #Se genera la petición a la página web, obtenemos el contenido en "formato html" de la página
    response = requests.get(url)
    content = response.text

    #Buscamos todos los elementos del contenido que tengan el tag: div y la clase: "footballbox"
    soup = BeautifulSoup(content, 'lxml')
    matches = soup.find_all('div', class_="footballbox")


    #Con lo anterior obtenemos el fixture completo de los torneos, y separamos los elementos correspondientes
    home = []
    score = []
    away = []

    #Rellenamos las listas con los elementos correspondientes en matches...
    for match in matches:
        home.append(match.find('th', class_="fhome").get_text())
        score.append(match.find('th', class_="fscore").get_text())
        away.append(match.find('th', class_="faway").get_text())
    
    #Con lo anterior, creamos un diccionario y, posteriormente, un DataFrame con los partidos y sus resultados
    dict_America = {'home':home, 
                    'score':score,
                    'away':away}

    df_America = pd.DataFrame(dict_America)
    df_America['year'] = year_A

    return df_America

In [8]:
get_matches(1916)

,home,score,away,year
0,Uruguay,4–0,Chile,1916
1,Argentina,6–1,Chile,1916
2,Brazil,1–1,Chile,1916
3,Argentina,1–1,Brazil,1916
4,Uruguay,2–1,Brazil,1916
5,Argentina,0–0[a],Uruguay,1916


In [9]:
#Finalmente, creamos una lista con el fixture de cada edición de la copa y luego lo unimos en un DataFrame


TotalMatches = [get_matches(year_A) for year_A in years_America]
df_TotalMatches = pd.concat(TotalMatches, ignore_index=True)

In [10]:
df_conmebol = df_TotalMatches[df_TotalMatches['year'] != 2024]
df_fixture = df_TotalMatches[df_TotalMatches['year'] == 2024]

## **Recolección de datos faltantes con Selenium**

In [11]:
#Previa revisión, sabemos que los años con los que tenemos problemas son 2011 y 2015
missing_years = [2011, 2015]

In [25]:
#WebScrapping con Selenium

#Creamos una petición de servicio y habilitamos el uso del driver de Chrome
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

#Definimos una función similar a get_maches, para los partidos faltantes
def get_missing_matches(year):

    url = f'https://en.wikipedia.org/wiki/{year}_Copa_America'

    driver.get(url) #Habilitamos el acceso al driver 

    #Buscamos los elementos html correspondientes a los partidos
    missing_matches = driver.find_elements(by='xpath', value='//tr[@style="font-size:90%"]')

    home = []
    score = []
    away = []

    for match in missing_matches:
        home.append(match.find_element(by='xpath', value='./td[1]').text)
        score.append(match.find_element(by='xpath', value='./td[2]').text)
        away.append(match.find_element(by='xpath', value='./td[3]').text)

    dict_missing = {'home':home, 
                    'score':score,
                    'away':away}

    df_missing = pd.DataFrame(dict_missing)
    df_missing['year'] = year
    time.sleep(1) #Tiempo de espera para pasar de una página a la otra

    return df_missing

#Creamos las listas con los partidos faltantes, para posteriormente crear un DataFrame
missing_data = [get_missing_matches(year) for year in missing_years]

driver.quit() #Línea necesaria para que el driver deje de controlar la página web

df_missing_data = pd.concat(missing_data, ignore_index=True)

## **Exportación de todos los datos**

In [21]:
with open('dict_tables', 'wb') as output:
    pickle.dump(dict_tables, output)

In [22]:
df_fixture.to_csv('Programacion_Copa_America_2024.csv', index=False)

In [23]:
df_conmebol.to_csv('Conmebol_Copa_America_initial_data.csv', index=False)

In [24]:
df_missing_data.to_csv('Conmebol_Copa_America_missing_data.csv', index=False)